In [27]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

trades = pd.read_json("./backtestOrderbook.json")
signals = pd.read_json("./signals.json")
busd = pd.read_json("./busdOverTime.json")

value = pd.read_json("./tradingWalletValueOverTime.json")
value = value.drop(value[value.timestamp == 0].index)

fig = go.Figure()
fig = make_subplots(specs=[[{"secondary_y": True}]])

asset = trades.asset.unique()[0]

valueAt0 = value[value.timestamp == 1].value.values[0]
priceAt0 = signals[(signals.timestamp == 1) & ((signals.asset == asset))].price.values[0]
z = valueAt0 / priceAt0

filteredSignals = signals[signals["asset"] == asset]
filteredTrades = trades[(trades["asset"] == asset)]

colmap = {
     "closed" : "limegreen",
     "stoploss" : "red",
     "opened" : "royalblue"
 }

barmap = {
     "BULL" : "limegreen",
     "BEAR" : "red",
     "RANGE" : "lightblue",
     "UNKNOWN" : "lightgrey"
 }

# AVAILABLE FUNDS
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
busd["valuescaled"] = scaler.fit_transform(busd["value"].values.reshape(-1,1))
fig.add_trace(
    go.Scatter(
        x=busd["timestamp"],
        y=busd["valuescaled"], mode = "lines",
        name = "BUSD liquid",
        marker = dict(color = "green")
    ), secondary_y = True)

# ASSET PRICE
fig.add_trace(
    go.Scatter(
        x=filteredSignals["timestamp"],
        y=filteredSignals["price"] * z, mode = "lines",
        name = "Price of " + asset + " (normalized)",
        marker = dict(color = "lightslategrey")
    ))



# VALUE OF WALLET
fig.add_trace(
    go.Scatter(
        x=value["timestamp"],
        y=value["value"], mode = "lines",
        name = "Wallet value over time",
        marker = dict(color = "orange")
    ), secondary_y = False)

# ORDERS
fig.add_trace(
        go.Scatter(
            x = filteredTrades["timestamp"],
            y = filteredTrades["price"] * z,
            mode = "markers",
            name = "opened",
            marker = dict(color = colmap["opened"])
        ))
for s in trades["status"].unique():
    filteredTrades = trades[(trades["asset"] == asset) & (trades["status"] == s)]
    if s != "open":
        fig.add_trace(
            go.Scatter(
                x= filteredTrades["closedPointInTime"],
                y= filteredTrades["closedAt"] * z,
                mode = "markers",
                name = s,
                marker = dict(
                        color = colmap[s]
                    )
            ))
filteredSignals["bar"] = max(filteredSignals["price"]) * z * 1.1

for b in filteredSignals["SIGNAL_ADX"].unique():
    filteredSignalsBars = filteredSignals[filteredSignals["SIGNAL_ADX"] == b]
    fig.add_trace(
            go.Bar(
                x = filteredSignalsBars["timestamp"],
                y = filteredSignalsBars["bar"],
                name = b,
                width = 0.4,
                marker = dict(color = barmap[b])),

            )
fig.update_yaxes(range = [min(filteredSignals["price"]) * z * 0.95,max(filteredSignals["price"]) * z * 1.05])
fig.update_xaxes(range = [min(filteredSignals["timestamp"]) -1, max(filteredSignals["timestamp"]) + 1])

# Secondary Axis update
fig.update_yaxes(range = [0, 1], secondary_y=True)

fig.show()

In [2]:
fig = go.Figure()
fig = make_subplots()

for a in signals.asset.unique():
    priceAt0 = signals[(signals.timestamp == 1) & ((signals.asset == a))].price.values[0]
    z = valueAt0 / priceAt0
    filteredSignals = signals[signals["asset"] == a]
    
    fig.add_trace(
        go.Scatter(
            x=filteredSignals["timestamp"],
            y=filteredSignals["price"] * z, mode = "lines",
            name = "Price of " + a + " (normalized)"
            
    ))
fig.add_trace(
    go.Scatter(
        x=value["timestamp"],
        y=value["value"], mode = "lines",
        name = "Wallet value over time",
        marker = dict(color = "orange")
    ), secondary_y = False)

fig.show()

In [9]:
0.0090 / 0.8500

0.010588235294117647

In [10]:
0.1104 / 21

0.005257142857142857

In [61]:
-4.06 / 2599.26

-0.0015619830259381514

In [40]:
import time
print(time.time())
delta = time.time() - (900000 * 500)
print(time.time() - delta)

1648634575.3766937
450000000.0


In [47]:
def printAndFetch(n):
    if (n - 500 >= 0):
        print(n)
        n -= 500
        printAndFetch(n)

In [48]:
n = 1300
printAndFetch(n)

1300
800
